In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (       
        ('printlog', False),
        ('rsiperiod',14),
        ('rsicrossover_high',50),
        ('rsicrossover_low',50)
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low
        self.dataopen = self.datas[0].open

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None


        # Add a MovingAverageSimple indicator
        self.rsi = bt.indicators.RelativeStrengthIndex(
            self.datas[0], period=self.params.rsiperiod)
        self.buysig = bt.And(self.datahigh[0] > self.dataclose[-1], self.rsi>self.params.rsicrossover_high)
        self.sellsig = bt.Or(self.rsi<self.params.rsicrossover_low, self.dataclose[0] < self.datalow[-1])


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.buysig:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.sellsig:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(RSI Period: %2d, RSI Crossover Lo/High: %2d/%2d) Ending Value %.2f' %
                 (self.params.rsiperiod, self.params.rsicrossover_low, self.params.rsicrossover_high, self.broker.getvalue()), doprint=True)

## Strategy Plot

In [ ]:
class FixedSlippageAndCommisionScheme(bt.CommInfoBase):
    '''Use this for both slippage and commissions together'''
    params = (
        ('commission', 12),
        ('slip_fixed ',0.01),
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_FIXED),
        )

    def _getcommission(self, size, price, pseudoexec):
        return self.p.commission#+self.p.slip_fixed

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

data = bt.feeds.PandasData(dataname = df['2016-01-01':'2018-01-01'])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Add Analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='TradeAnalyzer')

# Set our desired cash start
cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Alternatively, you can use:
# cerebro.broker.setcommission(commission=0.0)
# cerebro.broker.set_slippage_fixed(.00,  slip_open=True, slip_limit=True, slip_match=True, slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

sharpe=result[0].analyzers.sharpe.get_analysis()
drawdown=result[0].analyzers.drawdown.get_analysis()
trades=result[0].analyzers.TradeAnalyzer.get_analysis()                    
print('Sharpe Ratio:', sharpe)
print('Drawdown:',drawdown['moneydown'])
print('Trades:',trades['total'])

cerebro.plot(volume=False, iplot=True)

## In-sample Optimization

In [ ]:
class RSICrossoverAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.rsiperiod = self.strategy.params.rsiperiod
        self.rsicrossover_high = self.strategy.params.rsicrossover_high
        self.rsicrossover_low = self.strategy.params.rsicrossover_low
    
    def start(self):
        self.init_cash = self.strategy.broker.cash 
        self.ntrade = 0

    def next(self):
        pass

    def notify_trade(self, trade):
        self.ntrade += 1
    
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        
        outp = {
            'params': (self.rsiperiod, self.rsicrossover_high, self.rsicrossover_low),
            'profit': self.final_val - self.init_cash,
            'ntrade': self.ntrade
        }
        
        return outp

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    rsiperiod=range(6,15,2),rsicrossover_high=range(50,71,10),rsicrossover_low=range(30,51,10))

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Add Analyzers
cerebro.addanalyzer(RSICrossoverAnalyzer, _name='RSI')

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Run over everything
results = cerebro.run(maxcpus=1)

## Obtain Optimized Values using Analyzer

In [ ]:
# Optimize the variables for your selected timeframe for Net Profit
params  = []
n_trades = []
profit   = []
for res in results:
    r = res[0].analyzers.RSI.get_analysis()
    params.append(r['params'])
    n_trades.append(r['ntrade'])
    profit.append(r['profit'])
    
    

fig, (prof, nt) = plt.subplots(1,2, figsize=(15,5))
_=prof.hist(profit, bins=np.arange(-5000,20000,1000))
_=nt.hist(n_trades, bins=np.arange(0,500,25))

prof.set_title('Profit Distribution')
nt.set_title('# of Trades Distribution')
# Apply your best performing parameters for your out-of-sample period
prof_ind = np.argmax(profit) 
best_params = params[prof_ind]
print("Best Parameters: ",best_params,"\nTotal Profit: ", profit[prof_ind], "\nNumber of Trades: ", n_trades[prof_ind])

## Out-of-Sample

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy with best params from In Sample Data
cerebro.addstrategy(TestStrategy, 
                    rsiperiod=best_params[0],
                    rsicrossover_high=best_params[1],
                    rsicrossover_low=best_params[2])

# 20% out of sample data..6 months
data = bt.feeds.PandasData(dataname = df['2018-01-01':'2018-06-01'])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

# Add Analyzers
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='TradeAnalyzer')

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
out_results = cerebro.run()

out_sharpe=out_results[0].analyzers.sharpe.get_analysis()
out_drawdown=out_results[0].analyzers.drawdown.get_analysis()
out_trades=out_results[0].analyzers.TradeAnalyzer.get_analysis()                    
print('Sharpe Ratio:', out_sharpe)
print('Drawdown:',out_drawdown['moneydown'])
print('Trades:',out_trades['total'])
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.plot(volume=False, iplot=True)